# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [2]:
#include <stdio.h>

int main(){
    float S = 200;
    int N = 5;
    float J = 1;
    
    for(int i = 0; i < N; ++i){
        printf("Parcela %d eh: %0.2f\n", i+1, S);
        S += S*J/100;
    }
    return 0;
}

Parcela 1 eh: 200.00
Parcela 2 eh: 202.00
Parcela 3 eh: 204.02
Parcela 4 eh: 206.06
Parcela 5 eh: 208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [3]:
#include <stdio.h>

float proximaParcela(float parcela, int juros){
    parcela += parcela*juros/100;
    return parcela;
}

int main(){
    float S = 200;
    int N = 5;
    float J = 1;  
    
    for(int i = 0; i < N; ++i){
        printf("Parcela %d eh: %0.2f\n", i+1, S);
        S = proximaParcela(S, J);
    }
    return 0;
}

Parcela 1 eh: 200.00
Parcela 2 eh: 202.00
Parcela 3 eh: 204.02
Parcela 4 eh: 206.06
Parcela 5 eh: 208.12


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [8]:
#include <stdio.h>

float S;
int N ;
float J;

float inicializaEmprestimo(int numero_parcelas, float parcela, float juros){
    S = parcela;
    N = numero_parcelas;
    J = juros;
    return S;
}

float proximaParcela(){
    S += S*J/100;
    N = N-1;
    if(N > 0){
        return S;
    }
    else{
        return 0;
    }
}

int main(){
    float parcela;
    int parcela_atual = 1;
    parcela = inicializaEmprestimo(5,200,1);
    while(parcela != 0){
        printf("Parcela %d eh: %0.2f\n", parcela_atual, parcela);
        parcela = proximaParcela();
        parcela_atual += 1;
    }
    return 0;
}

Parcela 1 eh: 200.00
Parcela 2 eh: 202.00
Parcela 3 eh: 204.02
Parcela 4 eh: 206.06
Parcela 5 eh: 208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [10]:
#include <stdio.h>
#include "stdlib.h"

struct _dados{
    float parcela;
    float juros;
};

typedef struct _dados dados;

void proximaParcela(dados *estado){
    estado->parcela += estado->parcela*estado->juros/100;
}


int main(){
    float S = 200;
    int N = 5;
    float J = 1;
    
    dados *estado;
    estado = malloc(sizeof(dados));
    //testa-se se a alocação de memória foi bem sucedida
    if (estado == NULL) {
      exit(1);
    }
    estado->parcela = S;
    estado->juros = J;

    for(int i = 0; i < N; ++i){
        printf("Parcela %d eh: %0.2f\n", i+1, estado->parcela);
        proximaParcela(estado);
    }
    
    return 0;
}

Parcela 1 eh: 200.00
Parcela 2 eh: 202.00
Parcela 3 eh: 204.02
Parcela 4 eh: 206.06
Parcela 5 eh: 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [11]:
#include <stdio.h>
#include "stdlib.h"

typedef struct No{
    float juros;
    float parcela;
    int total_parcelas;
    int numero_parcelas;
    int emprestimo;
    struct No * prox;
}No;

//VARIÁVEl GLOBAl
No * raiz = NULL;

No* criarLista(){
    No* lista;
    lista = malloc(sizeof(No));
    lista->prox = NULL;
    return lista;
}

void adicionarEmprestimo(float S, int N, float J, int E){
    No * novo;
    No * aux;
    if(raiz == NULL){
        raiz = criarLista();
    }
    aux = raiz;
    while(aux->prox !=NULL){
        aux = aux->prox;
    }
    novo = malloc(sizeof(No));
    aux->prox = novo;
    novo->prox = NULL;
    novo->juros = J;
    novo->parcela = S;
    novo->total_parcelas = N;
    novo->numero_parcelas = 1;
    novo->emprestimo = E;
}

void excluirEmprestimo(int emprestimo, No ** raiz){
    No * ant;
    No * atual;
    No * post;
    ant = *raiz;
    post = (* raiz)->prox->prox;
    for(atual=(* raiz)->prox; atual!=NULL; ant= atual, atual = post, post = post->prox){
        if(atual->emprestimo == emprestimo){
            ant->prox = atual->prox;
            free(atual);
            break;
        }
    }
    if ((* raiz)->prox == NULL) {
       free(*raiz);
       * raiz = NULL;
    }
}

void proximaParcela(){
    No * aux;
    aux = raiz->prox;
    if( raiz == NULL){
        return;
    }
    while(aux != NULL){
        aux->parcela += aux->parcela*aux->juros/100;
        aux->numero_parcelas = aux->numero_parcelas + 1;
        if(aux->numero_parcelas > aux->total_parcelas){
            excluirEmprestimo(aux->emprestimo, &raiz);
        }
        aux= aux->prox;
    }
}

int imprimirParcela(){
    if( raiz == NULL){
        return -1;
    }
    for (No * aux = raiz->prox; aux != NULL; aux = aux->prox) {
        printf("Emprestimo %d: parcela %d eh %0.2f\n", aux->emprestimo, aux->numero_parcelas, aux->parcela);
    }
    return 0;
}


int main(){
    //Emprestimo 1
    adicionarEmprestimo(200, 5, 1, 1);
    //Emprestimo 2
    adicionarEmprestimo(500, 7, 2, 2);
    while(imprimirParcela() != -1){
        proximaParcela();
    }
    return 0;
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
